<a href="https://colab.research.google.com/github/Jinukki/KJU/blob/master/seq2seqReport.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 시퀀스-투-시퀀스(Sequence-to-Sequence, seq2seq)

##### 지금 이 공학전산응용을 듣는 학생들 중 거의 대부분은 조사를 할 때 혹은 과제를 할 때 구글이나 네이버의 번역기를 이용해 본 경험이 있을 것이다. 그 번역이 완전히 매끄러웠는지는 모르겠지만, 어쨌든 그럴싸한 순서를 가진 문장이 번역되어 나온다. 이런 문장들은 어떻게 번역되어 나온 것일까?

##### 자연어 처리 분야가 당면한 과제에서는 다양한 길이의 output 시퀀스가 중요할 때가 많다. 예를 들어 번역, 대화, 혹은 우리가 직접 글을 요약해야할 때 등이 있다. 이들은 모두 단어 한 개만 나오면 끝이 아니라 "문장" 즉, 단어들의 시퀀스가 나와줘야 평가가 가능하다. 이제 시퀀스-투-시퀀스(Sequence-to-Sequence), seq2seq를 알아보자. seq2seq는 입력된 시퀀스로부터 다른 도메인의 시퀀스를 출력하는 다양한 분야에서 사용되는 모델이다. 이 모델은 2014년 처음 사용되어 나온 지 5년 정도 밖에 되지 않았지만 기계 번역 분야에 있어 표준이 되어 가고 있다. 예를 들어 챗봇(Chatbot)과 기계 번역(Machine Translation)이 그러한 대표적인 예인데, 입력 시퀀스와 출력 시퀀스를 각각 질문과 대답으로 구성하면 챗봇으로 만들 수 있고, 입력 시퀀스와 출력 시퀀스를 각각 입력 문장과 번역 문장으로 만들면 번역기로 만들 수 있다. 앞에서 언급했듯이 내용 요약(Text Summarization), STT(Speech to Text) 등에서 쓰일 수 있다.

##### 이번 챕터에서는 기계 번역을 예제로 시퀀스-투-시퀀스를 설명한다. 앞으로는 줄여서 seq2seq이라는 이름으로 설명할 것이다. seq2seq에 대한 구조를 이해하고, 케라스(keras)를 통해 직접 구현해보자.

---
> ### seq2seq의 구조를 알아보자

##### 앞으로의 설명 방식은 내부가 보이지 않는 커다란 블랙 박스에서 점차적으로 확대해가는 방식으로 설명할 것이다. 참고로 여기서 설명하는 내용의 대부분은 딥러닝 입문 책에서의 RNN 챕터 발표에서 언급한 내용들이다. 단지 이것을 가지고 어떻게 조립했느냐에 따라서 seq2seq라는 구조가 만들어진다.

![그림1](https://user-images.githubusercontent.com/52277776/70078949-0b855d00-1647-11ea-91e2-b9e54a52c944.PNG)

##### 위 그림은 seq2seq 모델로 만들어진 번역기가 'I am a student'라는 영어 문장을 입력받아서, 'je suis étudiant'라는 프랑스 문장을 출력하는 모습을 보여준다. 그렇다면, seq2seq 모델 내부의 모습은 어떻게 구성되었을까?

![그림2](https://user-images.githubusercontent.com/52277776/70078957-0e804d80-1647-11ea-9c2c-5abaf163ba98.PNG)

##### seq2seq는 크게 두 개로 구성된 아키텍처로 구성되는데, 바로 인코더와 디코더이다. 인코더는 입력 문장의 모든 단어들을 순차적으로 입력받은 뒤에 마지막에 이 모든 단어 정보들을 압축해서 하나의 벡터로 만드는데, 이를 컨텍스트 벡터(context vector)라고 한다. 즉, 입력 시퀀스를 처리하고 자체 내부 상태를 반환한다. 이 상태는 다음 단계에서 디코더의 "context"또는 "conditioning"역할을 한다. 입력 문장의 정보가 하나의 컨텍스트 벡터로 모두 압축되면 인코더는 컨텍스트 벡터를 디코더로 전송한다. 디코더는 컨텍스트 벡터를 받아서 번역된 단어를 한 개씩 순차적으로 출력한다. 목표 시퀀스의 이전 문자가 주어지면 목표 시퀀스의 다음 문자를 예측하도록 훈련된다. 그런데 디코더의 훈련 과정에서는 이전 시점의 디코더 셀의 출력을 현재 시점의 디코더 셀의 입력으로 넣어주지 않고, 이전 시점의 '실제값'을 현재 시점의 디코더 셀의 입력값으로 하는 '교사 강요'라고 하는 방법을 사용한다. 그 이유는 이전 시점의 디코더 셀의 예측이 틀렸는데 이를 현재 시점의 디코더 셀의 입력으로 사용하면 현재 시점의 디코더 셀의 예측도 잘못될 가능성이 높고 이는 연쇄 작용으로 디코더 전체의 예측을 어렵게 하기 때문이다. 이런 상황이 반복되면 훈련 시간이 느려진다. 이를 막기 위해 이전 시점의 디코더 셀의 예측값 대신 실제값을 현재 시점의 디코더 셀의 입력으로 사용하는 방법을 사용한다. 이와 같이 RNN의 모든 시점에 대해서 이전 시점의 예측값 대신 실제값을 입력으로 주는 방법을 교사 강요라고 한다. 또한 중요하게, 인코더는 인코더로부터의 상태 벡터를 초기 상태로 사용하는데 이를 통해 디코더는 입력 시퀀스에 따라 생성해야 하는 정보를 배운다. 즉, t에서의 주어진 타겟 t+1의 타겟을 생성하도록 학습한다. 

![그림3](https://user-images.githubusercontent.com/52277776/70078960-104a1100-1647-11ea-808e-c944f81f1e92.PNG)

##### 컨텍스트 벡터에 대해서는 뒤에서 다시 언급하기로 하자. 위의 그림에서는 컨텍스트 벡터를 4의 사이즈로 표현하였지만, 실제로 사용되는 seq2seq 모델에서는 보통 수백 이상의 차원을 가지고 있다. 이제 인코더와 디코더의 내부를 좀 더 확대해 나가보자.

![seq2seq base 구조](https://user-images.githubusercontent.com/52277776/69826512-c92edb00-1256-11ea-91db-e8efb3123d99.PNG)

![그림4](https://user-images.githubusercontent.com/52277776/70078964-13dd9800-1647-11ea-96ca-948e695d68be.PNG)

##### 인코더 아키텍처와 디코더 아키텍처의 내부는 사실 두 개의 RNN 아키텍처이다. 입력 문장을 받는 RNN 셀을 인코더라고 하고, 출력 문장을 출력하는 RNN 셀을 디코더라고 하는데, 여기에서는 인코더의 RNN 셀을 주황색으로, 디코더의 RNN 셀을 초록색으로 표현할 것이다. 물론, 여기서 RNN 셀들은 LSTM을 기반으로 한 셀들이며 또는 GRU 셀들로 구성된다. 여기서 seq2seq의 장점을 언급하고 지나가자.

---
##### seq2seq의 장점은 이 모델이 사용하는 LSTM(Long-Short Term Memory)과의 연관성에서 나오는데, 과거의 번역 시스템이 장기 의존성(Long-Term Dependency)를 해결하기 구조였던 것에 비해, seq2seq는 전체 input을 훑어본 뒤에 임의의 어떤 시퀀스를 출력할 수 있으며, 효율적인 번역을 위해 input의 특정 부분만을 살펴볼 수도 있다.

##### 여기서 장기 의존성 문제에 설명을 하자면, RNN은 "The clouds are in the sky 문장에서 'sky'를 예측하는 경우" 와 같이 시간적으로 상관관계가 있는 데이터를 사용한다. 아래 그림과 같이 직진 데이터(t-1)과 현재 데이터(t) 간 상관관계를 고려하여 다음 데이터(t+1)를 예측하고자, 과거 데이터도 반영한 신경망 모델을 만든다.

![RNN구조](https://user-images.githubusercontent.com/52277776/69826516-cdf38f00-1256-11ea-923a-eb58b54d5d9c.PNG)

##### 그러나 RNN 처럼 직전 정보만 참고하는 것이 아니라,  그 전 정보를 고려해야 하는 경우(longer-term) 가 있다. <br>
 * 책을 읽을 때, 몇 페이지/챕터 전에 있는 정보를 머리 속에 기억하고 있어야 하는 경우<br>
 * I grew up in France... I speak fluent French. 문장에서 'French'를 예측하는 경우
##### 위의 예에서 마지막에 나온 French는 언어의 이름일 것이다. 그런데 French라는 단어의 의미가 많을텐데, French가 언어의 이름이라는 것을 어떻게 알 수 있을까? 바로 전에 나온 France라는 말에서 그 힌트를 찾을 수 있다. 두 예에서와 같이 전체적으로 앞에서 나온 정보를 고려해야 하는 경우, 문장 간의 간격(gap, 입력 위치의 차이)이 커질 수록, RNN은 두 두 정보의 맥락을 파악하기 어려워지는데 이를 해결하기 위해서 고안된 것이 LSTM이다. LSTM은 Tensorflow에서 구현 가능하다.

##### LSTM에 대한 설명은 발표에서 진행하였으므로 생략하고 seq2seq 이야기로 넘어가도록 하자.

---

##### 다시 본론으로 넘어와서, 우선 인코더를 자세히보면, 입력 문장은 단어 토큰화를 통해서 단어 단위로 쪼개지고 단어 토큰 각각은 RNN 셀의 각 시점의 입력이 된다. 인코더 RNN 셀은 모든 단어를 입력받은 뒤에 인코더 RNN 셀의 마지막 시점의 은닉 상태를 디코더 RNN 셀로 넘겨주는데 이를 컨텍스트 벡터라고 한다. 컨텍스트 벡터는 디코더 RNN 셀의 첫번째 은닉 상태로 사용된다. 디코더는 기본적으로 RNNLM(RNN Language Model)이다.

##### 디코더는 초기 입력으로 문장의 시작을 의미하는 심볼 <sos>가 들어간다. 디코더는 <sos>가 입력되면, 다음에 등장할 확률이 높은 단어를 예측한다. 첫번째 시점(time step)의 디코더 RNN 셀은 다음에 등장할 단어로 je를 예측하였다. 첫번째 시점의 디코더 RNN 셀은 예측된 단어 je를 다음 시점의 RNN 셀의 입력으로 입력한다. 그리고 두번째 시점의 디코더 RNN 셀은 입력된 단어 je로부터 다시 다음에 올 단어인 suis를 예측하고, 또 다시 이것을 다음 시점의 RNN 셀의 입력으로 보낸다. 디코더는 이런 식으로 기본적으로 다음에 올 단어를 예측하고, 그 예측한 단어를 다음 시점의 RNN 셀의 입력으로 넣는 행위를 반복한다. 이 행위는 문장의 끝을 의미하는 심볼인 <eos>가 다음 단어로 예측될 때까지 반복된다. 지금 설명하는 것은 테스트 과정 동안의 이야기이다.

##### seq2seq는 훈련 과정과 테스트 과정(또는 실제 번역기를 사람이 쓸 때)의 작동 방식이 조금 다르다. 훈련 과정에서는 디코더에게 인코더가 보낸 컨텍스트 벡터와 실제 정답인 상황인 <sos> je suis étudiant를 입력 받았을 때, je suis étudiant <eos>가 나와야 된다고 정답을 알려주면서 훈련한다. 이에 대해서는 뒤에 교사 강요(teacher forcing)를 설명하면서 다시 언급할 것이다. 반면 테스트 과정에서는 앞서 설명한 과정과 같이 디코더는 오직 컨텍스트 벡터와 <go>만을 입력으로 받은 후에 다음에 올 단어를 예측하고, 그 단어를 다음 시점의 RNN 셀의 입력으로 넣는 행위를 반복한다. 즉, 앞서 설명한 과정과 위의 그림은 테스트 과정에 해당된다. 이번에는 입, 출력에 쓰이는 단어 토큰들이 있는 부분을 좀 더 확대해보자.

![그림5](https://user-images.githubusercontent.com/52277776/70078967-163ff200-1647-11ea-9030-ade2345dea20.PNG)

##### 기계는 텍스트보다 숫자를 잘 처리한다. 그리고 자연어 처리에서 텍스트를 벡터로 바꾸는 임베딩을 발표에서 소개한 바 있다. 즉, seq2seq에서 사용되는 모든 단어들은 임베딩을 통해 임베딩 벡터로서 표현된 임베딩 벡터이다. 위 그림은 모든 단어에 대해서 임베딩 과정을 거치게 하는 단계인 임베딩 층(embedding layer)의 모습을 보여준다.

![그림6](https://user-images.githubusercontent.com/52277776/70078971-18a24c00-1647-11ea-88b6-3877aecf7b84.PNG)

##### 예를 들어 I, am, a, student라는 단어들에 대한 임베딩 벡터는 위와 같은 모습을 가진다. 여기서는 그림으로 표현하고자 사이즈를 4로 하였지만, 앞에서 언급하였듯이 실제 임베딩 벡터는 수백 개의 차원을 가질 수 있다. 이제 RNN 셀에 대해서 확대해보자.

##### 이미 RNN에 대해서 배운 적이 있지만, 다시 복습을 해보도록 하자. 하나의 RNN 셀은 각각의 시점(time step)마다 두 개의 입력을 받는다.

![그림7](https://user-images.githubusercontent.com/52277776/70078974-1b9d3c80-1647-11ea-8188-419a1191a450.PNG)

##### 현재 시점(time step)을 t라고 할 때, RNN 셀은 t-1에서의 은닉 상태와 t에서의 입력 벡터를 입력으로 받고, t에서의 은닉 상태를 만든다. 이때 t에서의 은닉 상태는 바로 위에 또 다른 은닉층이나 출력층이 존재할 경우에는 위의 층으로 보내거나, (사용할)필요가 없으면 값을 무시할 수 있다. 그리고 RNN 셀은 다음 시점에 해당하는 t+1의 RNN 셀의 입력으로 현재 t에서의 은닉 상태를 입력으로 보낸다.

##### RNN 챕터에서도 언급했지만, 이런 구조에서 현재 시점 t에서의 은닉 상태는 과거 시점의 동일한 RNN 셀에서의 모든 은닉 상태의 값들의 영향을 누적해서 받아온 값이라고 할 수 있다. 그렇기 때문에 앞서 우리가 언급했던 컨텍스트 벡터는 사실 인코더에서의 마지막 RNN 셀의 은닉 상태값을 말하는 것이며, 이는 입력 문장의 모든 단어 토큰들의 정보를 요약해서 담고있다고 할 수 있다.

##### 디코더는 인코더의 마지막 RNN 셀의 은닉 상태인 컨텍스트 벡터를 첫번째 은닉 상태의 값으로 사용한다. 디코더의 첫번째 RNN 셀은 이 첫번째 은닉 상태의 값과, 현재 t에서의 입력값인 <sos>로부터, 다음에 등장할 단어를 예측한다. 그리고 이 예측된 단어는 다음 시점인 t+1 RNN에서의 입력값이 되고, 이 t+1에서의 RNN 또한 이 입력값과 t에서의 은닉 상태로부터 t+1에서의 출력 벡터. 즉, 또 다시 다음에 등장할 단어를 예측하게 될 것이다. 이제 디코더가 다음에 등장할 단어를 예측하는 부분을 확대해보도록 하자.

![그림8](https://user-images.githubusercontent.com/52277776/70078977-1d670000-1647-11ea-9fd2-5f692b06a2ab.PNG)

##### 출력 단어로 나올 수 있는 단어들은 다양한 단어들이 있다. seq2seq 모델은 선택될 수 있는 모든 단어들로부터 하나의 단어를 골라서 예측해야 한다. 이를 예측하기 위해서 쓸 수 있는 함수로는 뭐가 있을까? 바로 소프트맥스 함수이다. 디코더에서 각 시점(time step)의 RNN 셀에서 출력 벡터가 나오면, 해당 벡터는 소프트맥스 함수를 통해 출력 시퀀스의 각 단어별 확률값을 반환하고, 디코더는 출력 단어를 결정한다.

##### 지금까지 가장 기본적인 seq2seq에 대해서 배워보았다. 사실 seq2seq는 어떻게 구현하느냐에 따라서 충분히 더 복잡해질 수 있다. 컨텍스트 벡터를 디코더의 초기 은닉 상태로만 사용할 수도 있고, 거기서 더 나아가 컨텍스트 벡터를 디코더가 단어를 예측하는 매 시점마다 하나의 입력으로 사용할 수도 있으며 거기서 더 나아가면 뒤에 소개할 어텐션 메커니즘이라는 방법을 통해 지금 알고있는 컨텍스트 벡터보다 더욱 문맥을 반영할 수 있는 컨텍스트 벡터를 구하여 매 시점마다 하나의 입력으로 사용할 수도 있다. 아래 그림은 더 구체적으로 seq2seq 모델의 그림이다.

![s2s 종합구조](https://user-images.githubusercontent.com/52277776/69826528-da77e780-1256-11ea-8373-23027af11129.PNG)

> ### 글자 레벨 기계 번역기(Character-Level Neural Machine Translation) 구현하기

##### 이제 seq2seq를 이용해서 기계 번역기를 만들어보도록 하자. 실제 성능이 좋은 기계 번역기를 구현하려면 정말 방대한 데이터가 필요하므로 여기서는 방금 배운 seq2seq를 실습해보는 수준에서 아주 간단한 기계 번역기를 구축해보도록 하자. 기계 번역기를 훈련시키기 위해서는 훈련 데이터로 병렬 코퍼스(parallel corpus)가 필요한데 여기서 병렬 코퍼스란, 두 개 이상의 언어가 병렬적으로 구성된 코퍼스를 의미한다. 이번 실습에서는 프랑스-영어 병렬 코퍼스를 사용할 것이다.

##### 우선 병렬 코퍼스 데이터에 대한 이해를 해보자. 병렬 데이터라고 하면 병렬 데이터는 쌍이 되는 모든 데이터가 길이가 같다고 생각할 수 있겠지만, 여기서는 쌍이 된다고 해서 길이가 같지 않다. 실제 번역기를 생각해보자. 구글 번역기에 '나는 학생이다.'라는 토큰의 개수가 2인 문장을 넣었을 때 'I am a student.'라는 토큰의 개수가 4인 문장이 나오는 것과 같은 이치이다. seq2seq는 기본적으로 입력 시퀀스와 출력 시퀀스의 길이가 다를 수 있다고 가정한다. 지금은 기계 번역기가 예제지만 seq2seq의 또 다른 유명한 예제 중 하나인 챗봇을 만든다고 가정해보면, 대답의 길이가 질문의 길이와 항상 똑같아야 한다고 하면 그 또한 이상하다.

##### 여기서 사용할 데이터는 영어 문장과 프랑스어 문장 사이에 탭으로 구분되는 구조가 하나의 샘플이다. 그리고 이와 같은 형식의 약 17만 개의 병렬 문장 샘플을 포함하고 있다. 앞으로의 코드에서 src는 source의 줄임말로 입력 문장을 나타내며, tar는 target의 줄임말로 번역하고자 하는 문장을 나타낸다.


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


##### 해당 파일을 구글 드라이브에 저장하였기에 마운트 작업을 통해 코랩과 드라이브를 연동시킨 후 진행하도록 한다.

In [0]:
import pandas as pd # 판다스를 통한 파일 읽기 및 확인
lines= pd.read_csv('/content/drive/My Drive/fra.txt', names=['src', 'tar', ' '], sep='\t')
len(lines)

170651

##### 전체 샘플의 개수는 총 약 17만 개이다.

##### 해당 데이터는 약 17만 개의 병렬 문장 샘플로 구성되어있지만 여기서는 간단히 6만 개의 샘플만 가지고 기계 번역기를 구축해보도록 하자. 우선 전체 데이터 중 6만 개의 샘플만 저장하고 현재 데이터가 어떤 구성이 되었는지 확인해보자.

In [0]:
lines = lines[0:60000] # 6만개 샘플만 저장
lines.sample(10) # 10개 샘플만 확인

,src,tar,
35334,I'm not wearing this.,Je ne vais pas porter ça.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
59855,I have some news for you.,J'ai des nouvelles pour vous.,CC-BY 2.0 (France) Attribution: tatoeba.org #1...
54147,I'm glad you came today.,Je suis contente que tu sois venue aujourd'hui.,CC-BY 2.0 (France) Attribution: tatoeba.org #3...
45276,Do you drink green tea?,Buvez-vous du thé vert ?,CC-BY 2.0 (France) Attribution: tatoeba.org #1...
5672,I'll clean up.,Je nettoierai.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
59226,How's married life going?,Comment va la vie de couple ?,CC-BY 2.0 (France) Attribution: tatoeba.org #3...
44341,Will that really work?,Est-ce que ça va vraiment fonctionner ?,CC-BY 2.0 (France) Attribution: tatoeba.org #5...
44626,You're not very funny.,Vous n'êtes pas très amusant.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
17052,You look perfect.,Vous avez l'air parfaite.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
4131,Look at this.,Regarde ça !,CC-BY 2.0 (France) Attribution: tatoeba.org #1...




##### 위의 테이블은 랜덤으로 선택된 10개의 샘플을 보여준다. 번역 문장에 해당되는 프랑스어 데이터는 앞서 배웠듯이 시작을 의미하는 심볼 <sos>과 종료를 의미하는 심볼 <eos>을 넣어주어야 한다. 여기서는 <sos>와 <eos> 대신 '\t'를 시작 심볼, '\n'을 종료 심볼로 간주하여 추가하고 다시 데이터를 출력해보자.

In [0]:
lines.tar = lines.tar.apply(lambda x : '\t '+ x + ' \n')
lines.sample(10)

,src,tar,
2331,I was drunk.,\t J'étais ivre. \n,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
21318,Were you obedient?,\t Avez-vous été obéissante ? \n,CC-BY 2.0 (France) Attribution: tatoeba.org #3...
45336,Do you like white wine?,\t Aimez-vous le vin blanc ? \n,CC-BY 2.0 (France) Attribution: tatoeba.org #4...
52777,I didn't do it that way.,\t Je ne l'ai pas fait comme ça. \n,CC-BY 2.0 (France) Attribution: tatoeba.org #5...
4712,You're stuck.,\t Vous êtes planté. \n,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
11197,I'm begging you.,\t Je t'en supplie. \n,CC-BY 2.0 (France) Attribution: tatoeba.org #3...
9905,Close that door.,\t Ferme cette porte ! \n,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
18114,He saw everything.,\t Il a tout vu. \n,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
11513,It's almost six.,\t C'est presque six heures. \n,CC-BY 2.0 (France) Attribution: tatoeba.org #5...
11623,Keep on smiling.,\t Restez souriants. \n,CC-BY 2.0 (France) Attribution: tatoeba.org #3...


##### 랜덤으로 10개의 샘플을 선택하여 출력하였다. 프랑스어 데이터에서 시작 심볼과 종료 심볼이 추가된 것을 볼 수 있다. 이제 글자 집합을 생성해보자. 단어 집합이 아니라 글자 집합이라고 하는 이유는 토큰 단위가 단어가 아니라 글자이기 때문이다.

In [0]:
# 글자 집합 구축
src_vocab=set()
for line in lines.src: # 1줄씩 읽음
    for char in line: # 1개의 글자씩 읽음
        src_vocab.add(char)

tar_vocab=set()
for line in lines.tar:
    for char in line:
        tar_vocab.add(char)

In [0]:
src_vocab_size = len(src_vocab)+1
tar_vocab_size = len(tar_vocab)+1
print(src_vocab_size)
print(tar_vocab_size)

79
106


##### 영어와 프랑스어는 각각 약 80개와 100개의 글자가 존재한다. 이 중에서 인덱스를 임의로 부여하여 일부만 출력해보자. 현 상태에서 인덱스를 사용하려고 하면 에러가 난다. 하지만 정렬하여 순서를 정해준 뒤에 인덱스를 사용하여 출력해주면 된다.

In [0]:
src_vocab = sorted(list(src_vocab))
tar_vocab = sorted(list(tar_vocab))
print(src_vocab[45:75])
print(tar_vocab[45:75])

['W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
['T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w']


##### 글자 집합에 글자 단위로 저장된 것을 확인할 수 있다. 이제 각 글자에 인덱스를 부여해보자.

In [0]:
src_to_index = dict([(word, i+1) for i, word in enumerate(src_vocab)])
tar_to_index = dict([(word, i+1) for i, word in enumerate(tar_vocab)])
print(src_to_index)
print(tar_to_index)

{' ': 1, '!': 2, '"': 3, '$': 4, '%': 5, '&': 6, "'": 7, ',': 8, '-': 9, '.': 10, '/': 11, '0': 12, '1': 13, '2': 14, '3': 15, '4': 16, '5': 17, '6': 18, '7': 19, '8': 20, '9': 21, ':': 22, '?': 23, 'A': 24, 'B': 25, 'C': 26, 'D': 27, 'E': 28, 'F': 29, 'G': 30, 'H': 31, 'I': 32, 'J': 33, 'K': 34, 'L': 35, 'M': 36, 'N': 37, 'O': 38, 'P': 39, 'Q': 40, 'R': 41, 'S': 42, 'T': 43, 'U': 44, 'V': 45, 'W': 46, 'X': 47, 'Y': 48, 'Z': 49, 'a': 50, 'b': 51, 'c': 52, 'd': 53, 'e': 54, 'f': 55, 'g': 56, 'h': 57, 'i': 58, 'j': 59, 'k': 60, 'l': 61, 'm': 62, 'n': 63, 'o': 64, 'p': 65, 'q': 66, 'r': 67, 's': 68, 't': 69, 'u': 70, 'v': 71, 'w': 72, 'x': 73, 'y': 74, 'z': 75, 'é': 76, '’': 77, '€': 78}
{'\t': 1, '\n': 2, ' ': 3, '!': 4, '"': 5, '$': 6, '%': 7, '&': 8, "'": 9, '(': 10, ')': 11, ',': 12, '-': 13, '.': 14, '0': 15, '1': 16, '2': 17, '3': 18, '4': 19, '5': 20, '6': 21, '7': 22, '8': 23, '9': 24, ':': 25, '?': 26, 'A': 27, 'B': 28, 'C': 29, 'D': 30, 'E': 31, 'F': 32, 'G': 33, 'H': 34, 'I': 3

# 이 이후부터 진행될 데이터의 추가적인 전처리 및 seq2seq 모델 구현에 대해서는 다음 중간보고서에 넣도록 하겠습니다.